<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment2/blob/main/Part_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np 
import argparse
import time
import imutils
import os

In [2]:
!pip install pytube

     |████████████████████████████████| 56 kB 537 kB/s 


In [3]:
from google.colab.patches import cv2_imshow
from pytube import YouTube

In [4]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pytube import YouTube
video = YouTube("https://www.youtube.com/watch?v=91VRyTvjoX4") #insert your video url

In [6]:
video.streams.all() #show all available formats

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  """Entry point for launching an IPython kernel.


[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="12fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="18" mime_type="video/mp4" res="360p" fps="24fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="22" mime_type="video/mp4" res="720p" fps="24fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="24fps" vcodec="avc1.640028" progressive="False" type="video">,
 <Stream: itag="248" mime_type="video/webm" res="1080p" fps="24fps" vcodec="vp9" progressive="False" type="video">,
 <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="24fps" vcodec="av01.0.08M.08" progressive="False" type="video">,
 <Stream: itag="136" mime_type="video/mp4" res="720p" fps="24fps" vcodec="avc1.4d401f" progressive="False" type="video">,
 <Stream: itag="247" mime_type="video/webm" res="720p" fps="24fps" vcodec="vp9" progressive="False

In [7]:
download = video.streams.filter(res="1080p") #download specific resolution
download

[<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="24fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="24fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="24fps" vcodec="av01.0.08M.08" progressive="False" type="video">]

In [8]:
download.first().download()

'/content/Selena Gomez - De Una Vez (Official Video).mp4'

In [9]:
ROOT_DIR = os.getcwd()

In [10]:

#Load yolo
def load_yolo():
	net = cv2.dnn.readNet("drive/MyDrive/DL-assignment2/yolov3-spp.weights", "drive/MyDrive/DL-assignment2/yolov3-spp.cfg")
	classes = []
	with open("drive/MyDrive/DL-assignment2/coco.names", "r") as f:
		classes = [line.strip() for line in f.readlines()] 
	
	output_layers = [layer_name for layer_name in net.getUnconnectedOutLayersNames()]
	colors = np.random.uniform(0, 255, size=(len(classes), 3))
	return net, classes, colors, output_layers

def load_image(img_path):
	# image loading
	img = cv2.imread(img_path)
	img = cv2.resize(img, None, fx=0.4, fy=0.4)
	height, width, channels = img.shape
	return img, height, width, channels


def display_blob(blob):
	'''
		Three images each for RED, GREEN, BLUE channel
	'''
	for b in blob:
		for n, imgb in enumerate(b):
			cv2.imshow(str(n), imgb)

def detect_objects(img, net, outputLayers):			
	blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
	net.setInput(blob)
	outputs = net.forward(outputLayers)
	return blob, outputs

def get_box_dimensions(outputs, height, width):
	boxes = []
	confs = []
	class_ids = []
	for output in outputs:
		for detect in output:
			scores = detect[5:]
			class_id = np.argmax(scores)
			conf = scores[class_id]
			if conf > 0.3:
				center_x = int(detect[0] * width)
				center_y = int(detect[1] * height)
				w = int(detect[2] * width)
				h = int(detect[3] * height)
				x = int(center_x - w/2)
				y = int(center_y - h / 2)
				boxes.append([x, y, w, h])
				confs.append(float(conf))
				class_ids.append(class_id)
	return boxes, confs, class_ids
			
def draw_labels(boxes, confs, colors, class_ids, classes, img): 
	indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
	font = cv2.FONT_HERSHEY_PLAIN
	for i in range(len(boxes)):
		if i in indexes:
			x, y, w, h = boxes[i]
			label = str(classes[class_ids[i]])
			color = colors[i]
			cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
			cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
	# cv2_imshow(img)

def image_detect(img_path): 
	model, classes, colors, output_layers = load_yolo()
	image, height, width, channels = load_image(img_path)
	blob, outputs = detect_objects(image, model, output_layers)
	boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
	draw_labels(boxes, confs, colors, class_ids, classes, image)
	while True:
		key = cv2.waitKey(1)
		if key == 27:
			break

def start_object_detection_in_video(video_path,output_file_name):
  model, classes, colors, output_layers = load_yolo()
  cap = cv2.VideoCapture(video_path)
  writer = None
  processed_frame_count = 0

  # try to determine the total number of frames in the video file
  try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
    else cv2.CAP_PROP_FRAME_COUNT
    total = int(cap.get(prop))
    print("[INFO] {} total frames in video".format(total))
  # an error occurred while trying to determine the total
  # number of frames in the video file
  except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

  while True:
    processed_frame_count += 1
    if(processed_frame_count %5 == 0):
      print("Processed frames:"+str(processed_frame_count))
    
    (grabbed, frame) = cap.read()
    if not grabbed:
      break
    height, width, channels = frame.shape
    blob, outputs = detect_objects(frame, model, output_layers)
    boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
    draw_labels(boxes, confs, colors, class_ids, classes, frame)
    # check if the video writer is None
    if writer is None:
		# initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      # outvid = os.path.join(ROOT_DIR, "out.mp4")
      writer = cv2.VideoWriter(output_file_name, fourcc, 30,
                               (frame.shape[1], frame.shape[0]), True)
      
	  # write the output frame to disk
    writer.write(frame)
  # release the file pointers
  print("[INFO] cleaning up...")
  writer.release()


In [27]:
def start_object_detection_on_youtube_video(video_link,download_video_file_name,output_file_name):
  video = YouTube(video_link)
  video_file_name = video.title
  resolutions_order = ["2160p","1080p","720p","480p","360p"]
  res_found = False
  count = 0
  while(not(res_found)):
    download = video.streams.filter(res=resolutions_order[count])
    if(len(download) >0):
      print("Video format which will be downloaded:"+str(download.first()))
      res_found = True
      download.first().download(filename=str(download_video_file_name)+".mp4" )
    count += 1
  start_object_detection_in_video(str(download_video_file_name)+".mp4",output_file_name)

In [32]:
def get_stream_filters_of_video(video_link):
  video = YouTube(video_link)
  return video.streams.all()

In [33]:
get_stream_filters_of_video("https://www.youtube.com/watch?v=qGCuCWDs1ts")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  This is separate from the ipykernel package so we can avoid doing imports until


[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="7fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">,
 <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="av01.0.08M.08" progressive="False" type="video">,
 <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">,
 <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False"

In [ ]:
start_object_detection_on_youtube_video(video_link="https://www.youtube.com/watch?v=qGCuCWDs1ts",download_video_file_name="yt_sharpest_kid",output_file_name="yt_sharpest_kid_detection.avi")
# start_object_detection_in_video("drive/MyDrive/DL-assignment2/bird.mp4",output_file_name="bird_detection.avi")

Video format which will be downloaded:<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">
[INFO] 2390 total frames in video
Processed frames:5
Processed frames:10
Processed frames:15
Processed frames:20
Processed frames:25
Processed frames:30
Processed frames:35
Processed frames:40
Processed frames:45
Processed frames:50
Processed frames:55
Processed frames:60
Processed frames:65
Processed frames:70
Processed frames:75
Processed frames:80
Processed frames:85
Processed frames:90
Processed frames:95
Processed frames:100
Processed frames:105
Processed frames:110
Processed frames:115
Processed frames:120
Processed frames:125
Processed frames:130
Processed frames:135
Processed frames:140
Processed frames:145
Processed frames:150
Processed frames:155
Processed frames:160
Processed frames:165
Processed frames:170
Processed frames:175
Processed frames:180
Processed frames:185
Processed frames:190
Processed frames:195
Processe